In [1]:
import pandas as pd

In [2]:
compounding_period={
    "Annually": 1,
    "Semi-Annually": 2,
    "Quarterly": 4, 
    "Monthly": 12
}

payment_periods = {
    "Annually": {"periods_per_year": 1, "months_offset": 12, "day_offset": 0},
    "Semi-Annually": {"periods_per_year": 2, "months_offset": 6, "day_offset": 0},
    "Quarterly": {"periods_per_year": 4, "months_offset": 3, "day_offset": 0},
    "Bi-Monthly": {"periods_per_year": 6, "months_offset": 2, "day_offset": 0},
    "Monthly": {"periods_per_year": 12, "months_offset": 1, "day_offset": 0},
    "Semi-Monthly": {"periods_per_year": 24, "months_offset": 0, "day_offset": 15},
    "Bi-Weekly": {"periods_per_year": 26, "months_offset": 0, "day_offset": 14},
    "Weekly": {"periods_per_year": 52, "months_offset": 0, "day_offset": 7}
}


In [3]:
def amortization(principal, amortization_term, interest, compounding_frequency, payment_frequency, cpr, mortgage_term=0):
    calculated_dict=dict()
    calculated_dict["compounding period"]=compounding_period[compounding_frequency]
    calculated_dict["periods per year"]=payment_periods[payment_frequency]["periods_per_year"]
    calculated_dict["interest rate per payment"] = ((1+(interest/calculated_dict["compounding period"]))**(calculated_dict["compounding period"]/calculated_dict["periods per year"]))-1
    if mortgage_term:
        calculated_dict["renewal period"] = (mortgage_term/12)*calculated_dict["periods per year"]
    else:
        calculated_dict["renewal period"] = 0
    calculated_dict["amortization period"] = (amortization_term/12)*calculated_dict["periods per year"]
    calculated_dict["payment per period"] = ((calculated_dict["interest rate per payment"])*principal)/(1-((1+(calculated_dict["interest rate per payment"]))**(-1*calculated_dict["amortization period"])))
    calculated_dict["smm"] = cpr/calculated_dict["periods per year"]
    calculated_dict["month offset"] = payment_periods[payment_frequency]["months_offset"]
    calculated_dict["day offset"] = payment_periods[payment_frequency]["day_offset"]
    return calculated_dict

In [4]:
from datetime import datetime, timedelta
from calendar import monthrange

def process_formula(period, month_offset, day_offset, date):
    # Convert date to a datetime object if it's a date string (assuming format "DD-MM-YYYY")
    date = datetime.strptime(date, "%d-%m-%Y") if isinstance(date, str) else date

    if period == "":
        return ""  # If period is empty, return an empty string
    
    if month_offset == 0:
        if day_offset == 15:
            if period % 2 == 0:  # If period is even
                # Add 1 month to date
                new_date = date.replace(day=1) + timedelta(days=32)  # Move to the next month
                new_date = new_date.replace(day=15)  # Keep the 15th day of the new month
            else:  # If period is odd
                new_date = date + timedelta(days=15)
        else:  # If a["day offset"] is not 15
            new_date = date + timedelta(days=day_offset)
    else:
        # Add C22 months to date
        new_month = date.month + month_offset
        new_year = date.year + (new_month - 1) // 12
        new_month = (new_month - 1) % 12 + 1
        # Get the last day of the new month to handle month-end overflow
        last_day = monthrange(new_year, new_month)[1]
        new_date = date.replace(year=new_year, month=new_month, day=min(date.day, last_day))

    # Return the new date as a string in "DD-MM-YYYY" format
    return new_date.strftime("%d-%m-%Y")#("%Y-%m-%d")

In [5]:
# a

In [6]:
import pandas as pd
from datetime import datetime, timedelta

# Input values
start_date = "13-01-2024"
original_principal = 10000  # Amount
amortization_term_months = 300  # Number of months for the amortization
mortgage_term_months = 12  # Mortgage term (should be same as renewal_period)
interest_rate = 0.05  # 5% annual interest rate
compounding_frequency = "Monthly"  # Compounded monthly
payment_frequency = "Weekly"  # payment intervals
cpr = 0.1  # 10% constant prepayment rate


a=amortization(principal=original_principal, amortization_term=amortization_term_months, mortgage_term=mortgage_term_months, interest=interest_rate, compounding_frequency=compounding_frequency, payment_frequency=payment_frequency, cpr=cpr)


compounding_period = a["compounding period"]  # Compounding frequency per year
periods_per_year = a["periods per year"]  # 12 periods per year
interest_rate_perpayment = a["interest rate per payment"]  # interest per period
renewal_period = a["renewal period"]  # Annual renewal
amortization_period = a["amortization period"]
payment_per_period = a["payment per period"]  # Fixed payment amount
smm = a["smm"]  #  (Single Monthly Mortality rate)
month_offset = a["month offset"]  # Payment month offset
day_offset = a["day offset"]  # Day offset

# Convert the start date to a datetime object
start_date = datetime.strptime(start_date, "%d-%m-%Y")

# Initialize an empty list to store the rows for the DataFrame
amortization_schedule = []

# Set the initial values for period 0
period = 0
date = start_date
opening_balance = original_principal
new_origination = original_principal
closing_balance = new_origination

# Loop over the periods
for period in range(int(a["amortization period"]) + 1):  # +1 to include period 0
    # Calculate the interest for the period
    interest = (opening_balance * interest_rate_perpayment)

    # Payment is the minimum of payment_per_period and the remaining balance (including interest)
    if opening_balance > payment_per_period:
        payment = (payment_per_period)
    else:
        payment = (opening_balance + interest)
    
    # Calculate principal paid for the period
    principal = payment - interest
    
    # Calculate prepayment
    if opening_balance - principal > 0:
        prepayment = (opening_balance * smm)
    else:
        prepayment = 0
    
    if not(period):
        opening_balance=0
        payment=0
        interest=0
        principal=0
        prepayment=0

    else:
        new_origination=0
        

    maturity=0
    # Apply prepayment, maturity, and calculate closing balance
    if period < renewal_period or mortgage_term_months==0:
        # For the first 'mortgage_term_months', no maturity (i.e., closing_balance is just reduced by prepayment and principal)
        closing_balance = opening_balance - prepayment - principal + new_origination
    elif period >= renewal_period or period>=amortization_period:
        # After mortgage term ends, apply maturity
        maturity = opening_balance - principal - prepayment # Remaining balance after regular payments
        closing_balance = 0  # After maturity, the loan is fully paid off
        
    
    # Prepare the data for this period
    row = {
        "period": period,
        "date": date,#.strftime("%d-%m-%Y"),
        "opening_balance": opening_balance,
        "payment": payment,
        "interest": interest,
        "principal": principal,
        "prepayment": prepayment,
        "new_origination": new_origination,
        "maturity": maturity if period >= mortgage_term_months else 0,
        "closing_balance": closing_balance
    }
    
    # Add the row to the amortization schedule
    amortization_schedule.append(row)
    
    # Update the opening balance for the next period
    opening_balance = closing_balance  # Opening balance of the next period is the closing balance of the current period
    
    # Increment the date
    # if payment_frequency=="Weekly":
    #     date = date + timedelta(days=7)
    # elif payment_frequency=="Bi-Weekly":
    #     date = date + timedelta(days=15)
    # elif payment_frequency=="Semi-Monthly":
    #     date = date + timedelta(days=15)
    # elif payment_frequency=="Monthly":
    #     date = date + timedelta(days=30)  # Adding 30 days to get the next month's date
    if (period >= renewal_period or period>=amortization_period) and mortgage_term_months!=0:
        break
    date = process_formula(period+1, a["month offset"], a["day offset"], date)

# Create the DataFrame from the amortization schedule list
df_amortization = pd.DataFrame(amortization_schedule)

# Display the resulting DataFrame
print(df_amortization)


    period                 date  opening_balance    payment  interest  \
0        0  2024-01-13 00:00:00         0.000000   0.000000  0.000000   
1        1           20-01-2024     10000.000000  13.468973  9.600013   
2        2           27-01-2024      9976.900271  13.468973  9.577837   
3        3           03-02-2024      9953.822789  13.468973  9.555683   
4        4           10-02-2024      9930.767532  13.468973  9.533550   
5        5           17-02-2024      9907.734479  13.468973  9.511438   
6        6           24-02-2024      9884.723608  13.468973  9.489348   
7        7           02-03-2024      9861.734899  13.468973  9.467278   
8        8           09-03-2024      9838.768330  13.468973  9.445230   
9        9           16-03-2024      9815.823879  13.468973  9.423204   
10      10           23-03-2024      9792.901526  13.468973  9.401198   
11      11           30-03-2024      9770.001248  13.468973  9.379214   
12      12           06-04-2024      9747.123025  1

In [7]:
a

{'compounding period': 12,
 'periods per year': 52,
 'interest rate per payment': 0.0009600013100943272,
 'renewal period': 52.0,
 'amortization period': 1300.0,
 'payment per period': 13.46897293717892,
 'smm': 0.0019230769230769232,
 'month offset': 0,
 'day offset': 7}

In [8]:
pd.DataFrame(list(a.items()), columns=['Key', 'Value'])

,Key,Value
0,compounding period,12.000000
1,periods per year,52.000000
2,interest rate per payment,0.000960
3,renewal period,52.000000
4,amortization period,1300.000000
5,payment per period,13.468973
6,smm,0.001923
7,month offset,0.000000
8,day offset,7.000000


In [9]:
df_amortization

,period,date,opening_balance,payment,interest,principal,prepayment,new_origination,maturity,closing_balance
0,0,2024-01-13 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,10000,0.000000,10000.000000
1,1,20-01-2024,10000.000000,13.468973,9.600013,3.868960,19.230769,0,0.000000,9976.900271
2,2,27-01-2024,9976.900271,13.468973,9.577837,3.891136,19.186347,0,0.000000,9953.822789
3,3,03-02-2024,9953.822789,13.468973,9.555683,3.913290,19.141967,0,0.000000,9930.767532
4,4,10-02-2024,9930.767532,13.468973,9.533550,3.935423,19.097630,0,0.000000,9907.734479
5,5,17-02-2024,9907.734479,13.468973,9.511438,3.957535,19.053336,0,0.000000,9884.723608
6,6,24-02-2024,9884.723608,13.468973,9.489348,3.979625,19.009084,0,0.000000,9861.734899
7,7,02-03-2024,9861.734899,13.468973,9.467278,4.001695,18.964875,0,0.000000,9838.768330
8,8,09-03-2024,9838.768330,13.468973,9.445230,4.023742,18.920708,0,0.000000,9815.823879
9,9,16-03-2024,9815.823879,13.468973,9.423204,4.045769,18.876584,0,0.000000,9792.901526


In [10]:
df_amortization.round(2).to_excel("forst.xlsx")

**DAILY SCHEDULE**

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Assuming 'periodic_amortization_df' is your periodic amortization table in pandas DataFrame
# Here's an example structure based on the table you provided:
# Columns: period, date, opening_balance, payment, interest, principal, prepayment, closing_balance

# Extract the date of the first payment and initialize the start date
#start_date = datetime.strptime("13-01-2024", "%d-%m-%Y")
df_amortization['date'] = pd.to_datetime(df_amortization['date'], format="%d-%m-%Y")

# Calculate daily interest rate
interest_rate_per_payment = 0.05 / 12  # Monthly rate, assuming monthly compounding
days_in_month = 30  # Adjust for actual days in month if needed, or calculate dynamically
daily_interest_rate = interest_rate_per_payment / days_in_month

# To store the daily amortization data
daily_amortization_records = []

# Iterate over each row of the periodic amortization table
for index, row in df_amortization.iterrows():
    # Get the payment date and details from the periodic table
    period=row["period"]
    payment_date = row['date']
    opening_balance = row['opening_balance']
    payment = row['payment']
    interest = row['interest']
    principal = row['principal']
    prepayment = row["prepayment"]
    new_origination = row["new_origination"]
    maturity=row["maturity"]
    closing_balance = row['closing_balance']
    
    # Calculate the interest for the days before the payment date
    current_date = start_date
    while current_date < payment_date:
        # Calculate daily interest based on the opening balance
        daily_interest = opening_balance * daily_interest_rate
        # Add daily record
        daily_amortization_records.append({
            "period" : period-1,
            'date': current_date,
            'opening_balance': opening_balance,
            'payment': 0,
            'interest': 0,#daily_interest,
            'principal': 0,
            'prepayment': 0,
            "new_origination": 0,
            'maturity': 0,
            'closing_balance': opening_balance
        })
        
        # Move to the next day
        current_date += timedelta(days=1)
    
    # Now, we have reached the payment day, record the payment details
    # Record the payment for the current day
    daily_amortization_records.append({
        'period' : period,
        'date': payment_date,
        'opening_balance': opening_balance,
        'payment': payment,
        'interest': interest,
        'principal': principal,
        'prepayment': prepayment,
        "new_origination": new_origination,
        "maturity": maturity,
        'closing_balance': closing_balance
    })
    
    # Update start_date for the next period
    start_date = payment_date + timedelta(days=1)

# Create a new DataFrame from the records
daily_amortization_df = pd.DataFrame(daily_amortization_records)

# Display or process your daily amortization table
print(daily_amortization_df)


     period       date  opening_balance    payment  interest  principal  \
0         0 2024-01-13         0.000000   0.000000  0.000000   0.000000   
1         0 2024-01-14     10000.000000   0.000000  0.000000   0.000000   
2         0 2024-01-15     10000.000000   0.000000  0.000000   0.000000   
3         0 2024-01-16     10000.000000   0.000000  0.000000   0.000000   
4         0 2024-01-17     10000.000000   0.000000  0.000000   0.000000   
..      ...        ...              ...        ...       ...        ...   
360      51 2025-01-07      8849.837397   0.000000  0.000000   0.000000   
361      51 2025-01-08      8849.837397   0.000000  0.000000   0.000000   
362      51 2025-01-09      8849.837397   0.000000  0.000000   0.000000   
363      51 2025-01-10      8849.837397   0.000000  0.000000   0.000000   
364      52 2025-01-11      8849.837397  13.468973  8.495855   4.973117   

     prepayment  new_origination     maturity  closing_balance  
0      0.000000            10000  

In [12]:
daily_amortization_df

,period,date,opening_balance,payment,interest,principal,prepayment,new_origination,maturity,closing_balance
0,0,2024-01-13,0.000000,0.000000,0.000000,0.000000,0.000000,10000,0.000000,10000.000000
1,0,2024-01-14,10000.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,10000.000000
2,0,2024-01-15,10000.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,10000.000000
3,0,2024-01-16,10000.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,10000.000000
4,0,2024-01-17,10000.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,10000.000000
...,...,...,...,...,...,...,...,...,...,...
360,51,2025-01-07,8849.837397,0.000000,0.000000,0.000000,0.000000,0,0.000000,8849.837397
361,51,2025-01-08,8849.837397,0.000000,0.000000,0.000000,0.000000,0,0.000000,8849.837397
362,51,2025-01-09,8849.837397,0.000000,0.000000,0.000000,0.000000,0,0.000000,8849.837397
363,51,2025-01-10,8849.837397,0.000000,0.000000,0.000000,0.000000,0,0.000000,8849.837397


In [13]:
daily_amortization_df["date"]=pd.to_datetime(daily_amortization_df["date"]).dt.strftime('%d-%m-%Y')

In [14]:
daily_amortization_df.round(2).to_excel("daily.xlsx")

In [15]:
# Convert 'date' column to datetime format
daily_amortization_df['date'] = pd.to_datetime(daily_amortization_df['date'], format='%d-%m-%Y')

# Group by year and month, aggregate the values for each month
df_monthly = daily_amortization_df.groupby(daily_amortization_df['date'].dt.to_period('M')).agg({
    'opening_balance': 'first',  # The opening balance is the first value in the month
    'payment': 'sum',            # Sum of payments in the month
    'interest': 'sum',           # Sum of interest in the month
    'principal': 'sum',          # Sum of principal in the month
    'prepayment': 'sum',         # Sum of prepayments in the month
    'new_origination': 'sum',    # Sum of new origination in the month
    'maturity': 'sum',           # Sum of maturity in the month
    'closing_balance': 'last'    # The closing balance is the last value in the month
}).reset_index()

# Convert period back to datetime format (e.g., '2024-02' to '01-02-2024')
df_monthly['date'] = df_monthly['date'].dt.to_timestamp()

# Display the result
print(df_monthly)

         date  opening_balance    payment   interest  principal  prepayment  \
0  2024-01-01         0.000000  26.937946  19.177850   7.760095   38.417116   
1  2024-02-01      9953.822789  53.875892  38.090018  15.785873   76.302016   
2  2024-03-01      9861.734899  67.344865  47.116125  20.228740   94.383134   
3  2024-04-01      9747.123025  53.875892  37.297436  16.578456   74.714313   
4  2024-05-01      9655.830256  53.875892  36.947378  16.928514   74.013075   
5  2024-06-01      9564.888667  67.344865  45.694003  21.650862   91.534336   
6  2024-07-01      9451.703469  53.875892  36.164661  17.711230   72.445136   
7  2024-08-01      9361.547102  67.344865  44.719840  22.625024   89.582891   
8  2024-09-01      9249.339187  53.875892  35.388703  18.487189   70.890735   
9  2024-10-01      9159.961263  53.875892  35.045987  18.829905   70.204205   
10 2024-11-01      9070.927153  67.344865  43.327547  24.017318   86.793846   
11 2024-12-01      8960.115990  53.875892  34.279688

In [16]:
df_monthly

,date,opening_balance,payment,interest,principal,prepayment,new_origination,maturity,closing_balance
0,2024-01-01,0.000000,26.937946,19.177850,7.760095,38.417116,10000,0.000000,9953.822789
1,2024-02-01,9953.822789,53.875892,38.090018,15.785873,76.302016,0,0.000000,9861.734899
2,2024-03-01,9861.734899,67.344865,47.116125,20.228740,94.383134,0,0.000000,9747.123025
3,2024-04-01,9747.123025,53.875892,37.297436,16.578456,74.714313,0,0.000000,9655.830256
4,2024-05-01,9655.830256,53.875892,36.947378,16.928514,74.013075,0,0.000000,9564.888667
5,2024-06-01,9564.888667,67.344865,45.694003,21.650862,91.534336,0,0.000000,9451.703469
6,2024-07-01,9451.703469,53.875892,36.164661,17.711230,72.445136,0,0.000000,9361.547102
7,2024-08-01,9361.547102,67.344865,44.719840,22.625024,89.582891,0,0.000000,9249.339187
8,2024-09-01,9249.339187,53.875892,35.388703,18.487189,70.890735,0,0.000000,9159.961263
9,2024-10-01,9159.961263,53.875892,35.045987,18.829905,70.204205,0,0.000000,9070.927153


In [17]:
df_monthly["date"]=pd.to_datetime(df_monthly["date"]).dt.strftime('%d-%m-%Y')

In [18]:
df_monthly.round(2).to_excel("monthly.xlsx")